<a href="https://colab.research.google.com/github/arnabbiswas66/multimodal-fake-news-classifier/blob/main/Twitter_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U "tensorflow-text==2.11.*"

In [ ]:
!pip install -q tf-models-official==2.11.0

In [3]:

import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
import re
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
tf.get_logger().setLevel('ERROR')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def get_df(file):
    return pd.read_csv(file,sep = '\t')

In [5]:
train_df = get_df('/content/drive/MyDrive/multimodal-news/mediaeval2015/devset/tweets.txt')
test_df = get_df('/content/drive/MyDrive/multimodal-news/mediaeval2015/testset/tweets.txt')

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14277 entries, 0 to 14276
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetId     14277 non-null  int64 
 1   tweetText   14277 non-null  object
 2   userId      14277 non-null  int64 
 3   imageId(s)  14277 non-null  object
 4   username    14277 non-null  object
 5   timestamp   14277 non-null  object
 6   label       14277 non-null  object
dtypes: int64(2), object(5)
memory usage: 780.9+ KB


In [11]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetId     3755 non-null   int64 
 1   tweetText   3755 non-null   object
 2   userId      3755 non-null   int64 
 3   imageId(s)  3755 non-null   object
 4   username    3755 non-null   object
 5   timestamp   3755 non-null   object
 6   label       3755 non-null   object
dtypes: int64(2), object(5)
memory usage: 205.5+ KB


In [14]:
#Changing 'humor' to 'fake'
train_df.loc[(train_df.label == 'humor'),'label'] = 'fake'
test_df.loc[(test_df.label == 'humor'),'label'] = 'fake'

In [15]:
#Removing retweets, reposts, and modified tweets
rtPattern1 = "(RT|rt|MT|mt|RP|rp):? @\w*:?"
rtPattern2 = "(\bRT\b|\brt\b|\bMT\b|\bmt\b|\bRP\b|\brp\b)"
rtPattern3 = "(@\w*:)"
rtPattern4 = "(#rt|#RT|#mt|#MT|#rp|#retweet|#Retweet|#modifiedtweet|#modifiedTweet|#ModifiedTweet|#repost|#Repost)"
rtPattern5 = "(via @\w*)"

retweets = train_df['tweetText'].str.contains(rtPattern1)
train_df = train_df[~retweets]

retweets = train_df['tweetText'].str.contains(rtPattern2)
train_df = train_df[~retweets]

retweets = train_df['tweetText'].str.contains(rtPattern3)
train_df = train_df[~retweets]

retweets = train_df['tweetText'].str.contains(rtPattern4)
train_df = train_df[~retweets]

retweets = train_df['tweetText'].str.contains(rtPattern5)
train_df = train_df[~retweets]

train_df.reset_index(drop=True, inplace=True)
train_df.shape

<ipython-input-15-fdd562f6ac27>:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retweets = train_df['tweetText'].str.contains(rtPattern1)
<ipython-input-15-fdd562f6ac27>:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retweets = train_df['tweetText'].str.contains(rtPattern2)
<ipython-input-15-fdd562f6ac27>:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retweets = train_df['tweetText'].str.contains(rtPattern3)
<ipython-input-15-fdd562f6ac27>:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  retweets = train_df['tweetText'].str.contains(rtPattern4)
<ipython-input-15-fdd562f6ac27>:20: UserWarning: This pattern is interpreted as a reg

(11550, 7)

In [16]:

#Removing remaining twitter handles @username
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: re.sub(r'@\w*', "", text))

In [17]:
#Removing emojis
emojis = re.compile("["
                    u"\U0001F600-\U0001F64F"
                    u"\U0001F300-\U0001F5FF"
                    u"\U0001F680-\U0001F6FF"
                    u"\U0001F1E0-\U0001F1FF"
                    u"\U00002702-\U000027B0"
                    u"\U000024C2-\U0001F251"
                    "]+", flags=re.UNICODE)

train_df['tweetText'] = train_df['tweetText'].apply(lambda text: emojis.sub(r'', text) if emojis.search(text) else text)

In [18]:
#Cleaning symbols - ampersand and newline
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: re.sub(r'&amp;|\\n', '', text))

In [19]:
#Removing urls
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: re.sub(r'http\S+', '', text))
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: re.sub(r'\\\/\S+', '', text))

In [21]:
#Removing whitespace
train_df['tweetText'] = train_df['tweetText'].apply(lambda text: " ".join(text.split()))

In [23]:
#Initialise stopwords

stopwords = nltk.corpus.stopwords.words()
stopwords.extend([':', ';', '[', ']', '"', "'", '(', ')', '.', '?', '#', '@', '...'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
#Removing stopwords
train_df['filteredTweet'] = train_df['tweetText'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))

In [27]:
train_df['label'].unique()

array(['fake', 'real'], dtype=object)

In [30]:
train_df['label'] = train_df['label'].apply(lambda x: 0 if x=='fake' else 1)

In [31]:
train_df['label'].unique()

array([0, 1])

In [32]:
final_train_df = train_df[['tweetText', 'label']]
final_train_df.head()

,tweetText,label
0,¿Se acuerdan de la película: “El día después d...,0
1,"Buena la foto del Huracán Sandy, me recuerda a...",0
2,Scary shit #hurricane #NY,0
3,My fave place in the world #nyc #hurricane #sa...,0
4,42nd #time #square #NYC #subway #hurricane,0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━